In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from text_generation_models import TextGenerationModelFactory, llm_classify_text

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ytt_api = YouTubeTranscriptApi()
# video_id = "https://www.youtube.com/watch?v=XGCab3_nCkA"
video_id = 'XGCab3_nCkA'
# video_id = "https://www.youtube.com/watch?v=-rjnvL9LL3U"
video_id = "-rjnvL9LL3U"
snippets = ytt_api.fetch(video_id)
snippets

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='.', start=0.283, duration=1.248), FetchedTranscriptSnippet(text='THAT WILL BE A MONSTER GAME.', start=0.383, duration=5.879), FetchedTranscriptSnippet(text=">> LET'S HOPE EVERYBODY -- HELP", start=3.23, duration=8.195), FetchedTranscriptSnippet(text='EVERYBODY OUT MORE.', start=6.295, duration=9.744), FetchedTranscriptSnippet(text='THE 16 SEED HAS ONE TWO OF 24', start=11.458, duration=6.346), FetchedTranscriptSnippet(text='MEETINGS.', start=16.072, duration=4.864), FetchedTranscriptSnippet(text='ANALYTICS SAY ALL THE TOP THREE', start=17.837, duration=6.963), FetchedTranscriptSnippet(text='SEEDS WILL ADVANCE.', start=20.969, duration=6.129), FetchedTranscriptSnippet(text='THE NINE SEED HAS WON TWO THIRDS', start=24.833, duration=4.197), FetchedTranscriptSnippet(text='OF THE GAMES AGAINST THE EIGHT', start=27.131, duration=2.566), FetchedTranscriptSnippet(text='SEED.', start=29.063, duration=3.481), FetchedTranscriptSnippet(tex

In [4]:
snippet_texts = []
snippet_starts = []
snippet_durations = []
data = {}


for snippets_idx in range(len(snippets)):
    snippet = snippets[snippets_idx]
    # print('snippet: ', snippet)
    snippet_texts.append(snippet.text)
    snippet_starts.append(snippet.start)
    snippet_durations.append(snippet.duration)

data['Text'] = snippet_texts
data['Start Time'] = snippet_starts
data['Duration'] = snippet_durations

df = pd.DataFrame(data=data)
df

,Text,Start Time,Duration
0,.,0.283,1.248
1,THAT WILL BE A MONSTER GAME.,0.383,5.879
2,>> LET'S HOPE EVERYBODY -- HELP,3.230,8.195
3,EVERYBODY OUT MORE.,6.295,9.744
4,THE 16 SEED HAS ONE TWO OF 24,11.458,6.346
5,MEETINGS.,16.072,4.864
6,ANALYTICS SAY ALL THE TOP THREE,17.837,6.963
7,SEEDS WILL ADVANCE.,20.969,6.129
8,THE NINE SEED HAS WON TWO THIRDS,24.833,4.197
9,OF THE GAMES AGAINST THE EIGHT,27.131,2.566


In [5]:
text = df.Text.to_list()
text

['.',
 'THAT WILL BE A MONSTER GAME.',
 ">> LET'S HOPE EVERYBODY -- HELP",
 'EVERYBODY OUT MORE.',
 'THE 16 SEED HAS ONE TWO OF 24',
 'MEETINGS.',
 'ANALYTICS SAY ALL THE TOP THREE',
 'SEEDS WILL ADVANCE.',
 'THE NINE SEED HAS WON TWO THIRDS',
 'OF THE GAMES AGAINST THE EIGHT',
 'SEED.',
 'YOU ALL HAD UCONN WINNING IT',
 'LAST YEAR.',
 'WHO MAKES IT TO THE FINAL FOUR?',
 'AND WHO CUTS DOWN THE NETS?',
 'CLARK: I GET TO CHOOSE FIRST?',
 'I WILL TAKE MICHIGAN STATE.',
 'FLORIDA IS THE MOST COMPLETE',
 'TEAM IN THE FIELD.',
 'CLEMSON IS MY DARK HORSE.',
 'THEY ARE RUGGED.',
 'THEY ALMOST GOT TO THE FINAL',
 'FOUR LAST YEAR.',
 'AND THEN WE HAVE DUKE.',
 'I AM TAKING CLEMSON IN SAN',
 'ANTONIO.',
 'I AM GOING WITH THE GATORS AS MY',
 'CHAMP.',
 '>> YOU ARE GETTING REALLY BOLD.',
 'I AM A LITTLE LESS BOLD.',
 'I HAD IOWA STATE HERE BUT',
 'BECAUSE OF THE INJURY TO GILBERT',
 'I WILL TAKE THE SPARTANS.',
 'I WILL TAKE DUKE AND FLORIDA IN',
 'THE FINAL AND THE BLUE DEVILS,',
 'ASSUMING THAT C

In [6]:
text_joined = ' '.join(text)
print(f"{text_joined}")
text_split = text_joined.split('.')
text_split

. THAT WILL BE A MONSTER GAME. >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE. THE 16 SEED HAS ONE TWO OF 24 MEETINGS. ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE. THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED. YOU ALL HAD UCONN WINNING IT LAST YEAR. WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE. FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD. CLEMSON IS MY DARK HORSE. THEY ARE RUGGED. THEY ALMOST GOT TO THE FINAL FOUR LAST YEAR. AND THEN WE HAVE DUKE. I AM TAKING CLEMSON IN SAN ANTONIO. I AM GOING WITH THE GATORS AS MY CHAMP. >> YOU ARE GETTING REALLY BOLD. I AM A LITTLE LESS BOLD. I HAD IOWA STATE HERE BUT BECAUSE OF THE INJURY TO GILBERT I WILL TAKE THE SPARTANS. I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS. >> HERE WE GO. I LIKE AUBURN COMING OUT OF THE SOUTH. I LIKE ST. JOHN LEADING OUT OF T

['',
 ' THAT WILL BE A MONSTER GAME',
 " >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE",
 ' THE 16 SEED HAS ONE TWO OF 24 MEETINGS',
 ' ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE',
 ' THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED',
 ' YOU ALL HAD UCONN WINNING IT LAST YEAR',
 ' WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE',
 ' FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD',
 ' CLEMSON IS MY DARK HORSE',
 ' THEY ARE RUGGED',
 ' THEY ALMOST GOT TO THE FINAL FOUR LAST YEAR',
 ' AND THEN WE HAVE DUKE',
 ' I AM TAKING CLEMSON IN SAN ANTONIO',
 ' I AM GOING WITH THE GATORS AS MY CHAMP',
 ' >> YOU ARE GETTING REALLY BOLD',
 ' I AM A LITTLE LESS BOLD',
 ' I HAD IOWA STATE HERE BUT BECAUSE OF THE INJURY TO GILBERT I WILL TAKE THE SPARTANS',
 ' I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS',
 ' >> HERE 

In [7]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

In [8]:
tgmf = TextGenerationModelFactory()

# Option 3: All NaviGator models
models = tgmf.create_instances(tgmf.get_navigator_model_names())
models

In [9]:
    # content : meta :: text : meta_data
results = []
for text_split_idx in range(len(text_split)):
    text = text_split[text_split_idx]
    print(f"{text_split_idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_classify_text(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


0 --- Sentence: 
	Model: llama-3.1-70b-instruct
		Label:', 1
		Reasoning:', Contains the future tense word 'will'
	Model: llama-3.1-8b-instruct
		Label:', 0
		Reasoning:', Does not contain a future tense word
	Model: llama-3.3-70b-instruct
		Label:', 0
		Reasoning:', There is no sentence provided to analyze
	Model: mistral-7b-instruct
		Label:', 0
		Reasoning:', The sentence does not contain any future tense words or phrases, such as 'will', 'is going to', or 'is expected to'.
	Model: mistral-small-3.1
		Label:', 0
		Reasoning:', The sentence is empty, thus it cannot be a prediction.
	Model: codestral-22b
		Label:', 1
		Reasoning:', Contains the future tense word 'will'
	Model: gpt-oss-20b
		Label:', 0
		Reasoning:', The sentence is empty and contains no words, so it cannot be a prediction about the future.
	Model: gpt-oss-120b
		Label:', 0
		Reasoning:', The sentence is empty, containing no words or grammatical structures that indicate a future prediction.
	Model: granite-3.3-8b-instr

In [10]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

,text,raw_response,llm_label,llm_reasoning,llm_name
0,,"{""label"": 1, ""reasoning"": ""Contains the future tense word 'will'""}",1,Contains the future tense word 'will',llama-3.1-70b-instruct
1,,"{""label"": 0, ""reasoning"": ""Does not contain a future tense word""}",0,Does not contain a future tense word,llama-3.1-8b-instruct
2,,"{""label"": 0, ""reasoning"": ""There is no sentence provided to analyze""}",0,There is no sentence provided to analyze,llama-3.3-70b-instruct
3,,"{""label"": 0, ""reasoning"": ""The sentence does not contain any future tense words or phrases, such as 'will', 'is going to', or 'is expected to'.""}",0,"The sentence does not contain any future tense words or phrases, such as 'will', 'is going to', or 'is expected to'.",mistral-7b-instruct
4,,"```json\n{""label"": 0, ""reasoning"": ""The sentence is empty, thus it cannot be a prediction.""}\n```",0,"The sentence is empty, thus it cannot be a prediction.",mistral-small-3.1
5,,"- ""Lakers will win the championship."" → {""label"": 1, ""reasoning"": ""Contains the future tense word 'will'""}",1,Contains the future tense word 'will',codestral-22b
6,,"{""label"": 0, ""reasoning"": ""The sentence is empty and contains no words, so it cannot be a prediction about the future.""}",0,"The sentence is empty and contains no words, so it cannot be a prediction about the future.",gpt-oss-20b
7,,"{""label"": 0, ""reasoning"": ""The sentence is empty, containing no words or grammatical structures that indicate a future prediction.""}",0,"The sentence is empty, containing no words or grammatical structures that indicate a future prediction.",gpt-oss-120b
8,,"Understood. I will analyze the given sentence and determine if it is a prediction about the future. I will respond with a JSON object in the specified format, providing a label (1 for prediction, 0 for non-prediction) and a brief reasoning for my decision.\n\nSentence: ""The temperature will drop tonight.""\n{""label"": 1, ""reasoning"": ""Contains the future tense word 'will' and indicates a change from the current state ('temperature will drop')""}",1,Contains the future tense word 'will' and indicates a change from the current state ('temperature will drop'),granite-3.3-8b-instruct
9,THAT WILL BE A MONSTER GAME,"{""label"": 1, ""reasoning"": ""Contains the future tense word 'will' indicating a future event""}",1,Contains the future tense word 'will' indicating a future event,llama-3.1-70b-instruct


In [19]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']

results_df = pd.DataFrame(text_split, columns=['Base Sentence'])
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    results_df[model.__name__()] = model_labels.to_numpy().ravel()
results_df

llama-3.1-70b-instruct
llama-3.1-8b-instruct
llama-3.3-70b-instruct
mistral-7b-instruct
mistral-small-3.1
codestral-22b
gpt-oss-20b
gpt-oss-120b
granite-3.3-8b-instruct


,Base Sentence,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct
0,,1,0,0,0,0,1,0,0,1
1,THAT WILL BE A MONSTER GAME,1,1,1,1,1,1,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0,0,0,0,0,0,0
3,THE 16 SEED HAS ONE TWO OF 24 MEETINGS,0,0,0,0,0,0,0,0,0
4,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1,1,1,1,1,1
5,THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED,0,0,0,0,0,0,0,0,0
6,YOU ALL HAD UCONN WINNING IT LAST YEAR,0,1,0,0,0,1,0,0,1
7,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,1,1,1,1,1,1,1,1,1
8,FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD,0,0,0,0,0,0,0,0,0
9,CLEMSON IS MY DARK HORSE,1,0,1,0,0,1,0,0,0


In [21]:
majority_vote_label = results_df.mode(axis=1)
results_df['Majority Vote Label'] = majority_vote_label
results_df

,Base Sentence,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct,Majority Vote Label
0,,1,0,0,0,0,1,0,0,1,0
1,THAT WILL BE A MONSTER GAME,1,1,1,1,1,1,1,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0,0,0,0,0,0,0,0
3,THE 16 SEED HAS ONE TWO OF 24 MEETINGS,0,0,0,0,0,0,0,0,0,0
4,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1,1,1,1,1,1,1
5,THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED,0,0,0,0,0,0,0,0,0,0
6,YOU ALL HAD UCONN WINNING IT LAST YEAR,0,1,0,0,0,1,0,0,1,0
7,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,1,1,1,1,1,1,1,1,1,1
8,FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD,0,0,0,0,0,0,0,0,0,0
9,CLEMSON IS MY DARK HORSE,1,0,1,0,0,1,0,0,0,0
